<a href="https://colab.research.google.com/github/oluwafemidiakhoa/MLprject/blob/main/Multiagent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install openai asyncio numpy pandas


In [8]:
import os

os.environ['OPENAI_API_KEY'] = ""




In [9]:
import os
import uuid
from openai import OpenAI

# Initialize the OpenAI client
client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Define the Agent class
class Agent:
    def __init__(self, agent_id=None):
        self.agent_id = agent_id or str(uuid.uuid4())
        self.role = 'Generalist'
        self.memory = []
        self.peers = []
        self.knowledge_base = None  # Reference to shared knowledge base
        self.communication_channel = None  # Reference to communication channel

    def perceive(self, environment):
        # Implement perception logic if needed
        pass

    def decide_role(self, task):
        prompt = f"""
You are Agent {self.agent_id}, analyzing the task: '{task}'.
Propose a role that you can assume to contribute effectively.
Explain your reasoning.
"""
        response = self.query_gpt(prompt)
        self.role = self.extract_role(response)
        return self.role

    def act(self, task):
        prompt = f"""
As {self.role}, perform the following task: '{task}'.
Provide your output and reasoning.
"""
        response = self.query_gpt(prompt)
        self.memory.append({'task': task, 'response': response})
        return response

    def reflect(self):
        last_action = self.memory[-1] if self.memory else None
        if last_action:
            prompt = f"""
Reflect on your recent action: '{last_action['task']}'.
Your response was: '{last_action['response']}'.
What went well? What could be improved?
"""
            reflection = self.query_gpt(prompt)
            self.memory.append({'reflection': reflection})
            return reflection

    def communicate(self, message, recipient_id=None):
        if self.communication_channel:
            self.communication_channel.send(self.agent_id, recipient_id, message)

    def receive_messages(self):
        if self.communication_channel:
            return self.communication_channel.receive(self.agent_id)
        return []

    def query_gpt(self, prompt):
        response = client.chat.completions.create(
            model='gpt-4',
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return response.choices[0].message.content

    def extract_role(self, response):
        # Simple parsing to extract the proposed role from the response
        lines = response.strip().split('\n')
        for line in lines:
            if 'Role' in line or 'role' in line:
                return line.split(':')[-1].strip()
        # Default role if none found
        return 'Generalist'

# Define the KnowledgeBase class
class KnowledgeBase:
    def __init__(self):
        self.data = {}

    def store(self, key, value):
        self.data[key] = value

    def retrieve(self, key):
        return self.data.get(key, None)

    def search(self, query):
        # Implement a simple search
        results = {k: v for k, v in self.data.items() if query.lower() in k.lower()}
        return results

# Define the CommunicationChannel class
class CommunicationChannel:
    def __init__(self):
        self.messages = []

    def send(self, sender_id, recipient_id, content):
        message = {'sender': sender_id, 'recipient': recipient_id, 'content': content}
        self.messages.append(message)

    def receive(self, agent_id):
        agent_messages = [
            msg for msg in self.messages
            if msg['recipient'] == agent_id or msg['recipient'] is None
        ]
        # Remove the messages once received
        self.messages = [msg for msg in self.messages if msg not in agent_messages]
        return agent_messages

# Initialize shared components
knowledge_base = KnowledgeBase()
communication_channel = CommunicationChannel()

# Create agents
agents = [Agent() for _ in range(5)]  # Create 5 agents for testing

# Assign shared components to agents
for agent in agents:
    agent.knowledge_base = knowledge_base
    agent.communication_channel = communication_channel
    agent.peers = [a.agent_id for a in agents if a.agent_id != agent.agent_id]

# Define the task
task = "Produce a comprehensive report on the latest advancements in renewable energy technologies."

# Role assignment function
def role_assignment(agents, task):
    role_proposals = {}
    for agent in agents:
        proposed_role = agent.decide_role(task)
        role_proposals[agent.agent_id] = proposed_role
        print(f"Agent {agent.agent_id} proposes role: {proposed_role}")

    # Handle potential role conflicts (simplified for testing)
    assigned_roles = {}
    roles_assigned = set()
    for agent_id, role in role_proposals.items():
        if role not in roles_assigned:
            assigned_roles[agent_id] = role
            roles_assigned.add(role)
        else:
            # If role conflict, assign a modified role
            assigned_roles[agent_id] = role + ' Assistant'
            roles_assigned.add(role + ' Assistant')

    # Update agents with assigned roles
    for agent in agents:
        agent.role = assigned_roles[agent.agent_id]
        print(f"Agent {agent.agent_id} assigned role: {agent.role}")

# Run role assignment
role_assignment(agents, task)

# Agents perform actions
for agent in agents:
    print(f"\nAgent {agent.agent_id} ({agent.role}) is acting.")
    action_output = agent.act(task)
    print(f"Output:\n{action_output}")

    # Self-reflection
    reflection = agent.reflect()
    print(f"Reflection:\n{reflection}")

    # Agents communicate their results to peers
    agent.communicate(f"My output: {action_output}")

# Agents receive messages
for agent in agents:
    messages = agent.receive_messages()
    print(f"\nAgent {agent.agent_id} received messages:")
    for message in messages:
        print(f"From Agent {message['sender']}: {message['content']}")

# Collect and integrate outputs
final_outputs = []
for agent in agents:
    if agent.memory:
        # Assuming the action output is stored before reflection
        last_action = agent.memory[-2] if len(agent.memory) >= 2 else agent.memory[-1]
        final_outputs.append(last_action['response'])

final_report = "\n---\n".join(final_outputs)
print("\nFinal Integrated Report:")
print(final_report)


Agent 76a40b9b-e7ab-43bc-ab29-38335735733b proposes role: Renewable Energy Research Analyst
Agent 65616a3f-07d1-47ed-9288-c704232712be proposes role: Renewable Energy Technology Research Analyst
Agent e3ac52d9-88c6-4559-abb5-79955da0458a proposes role: Renewable Energy Research Analyst
Agent 5d401753-4286-43da-a271-720e00e0b41b proposes role: Renewable Energy Research Analyst
Agent d0ab6db2-6e83-4934-96fd-c415b649acfe proposes role: Renewable Energy Research Analyst
Agent 76a40b9b-e7ab-43bc-ab29-38335735733b assigned role: Renewable Energy Research Analyst
Agent 65616a3f-07d1-47ed-9288-c704232712be assigned role: Renewable Energy Technology Research Analyst
Agent e3ac52d9-88c6-4559-abb5-79955da0458a assigned role: Renewable Energy Research Analyst Assistant
Agent 5d401753-4286-43da-a271-720e00e0b41b assigned role: Renewable Energy Research Analyst Assistant
Agent d0ab6db2-6e83-4934-96fd-c415b649acfe assigned role: Renewable Energy Research Analyst Assistant

Agent 76a40b9b-e7ab-43bc-ab